In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix 
import warnings as ws
ws.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/kaggle/input/passenger-list-for-the-estonia-ferry-disaster/estonia-passenger-list.csv")

In [ ]:
df.head()

In [ ]:
df.drop(columns = "PassengerId", inplace = True)
def clean_name(name):
    return name.lower().strip().replace(" ", "_")
df.rename(columns = clean_name, inplace =True)
df.head()

In [ ]:
X = df.drop(columns = "survived")
y = df.survived

In [ ]:
# Performing Some EDA to get more insights 
frame = df.copy()

In [ ]:
frame.head()

In [ ]:
# Checking for null values 
df.isna().sum()

In [ ]:
frame.dtypes

In [ ]:
# As the firstname and lastname are the unique values for the every person it doesn't gives us any significancef
frame.drop(columns  = ["firstname", "lastname"], inplace = True)

In [ ]:
frame.head()

In [ ]:
sns.set()
plt.figure(figsize = (9,9))
plt.title("Coutrywise passengers count", fontdict={'fontsize' : 20})
sns.countplot(y = frame.country, palette="plasma")
plt.show()

### Looks like the swedish people are present on the deck as compared to other people. we have  to deal with this country column wisely as it bears more no. of catagroies we will handle it carefully

In [ ]:
len(list(frame.country.unique()))

In [ ]:
frame.sex.replace({'M': 1, 'F' : 0 }, inplace = True)

In [ ]:
frame.head()

In [ ]:
frame["is_passenger"] = pd.get_dummies(df.category, drop_first= True).rename(columns= {'P': "is_passenger"})

In [ ]:
frame.country.value_counts()

In [ ]:
frame.head()
temp = pd.get_dummies(frame, drop_first = True)

In [ ]:
sns.set()
plt.figure(figsize = (20,20))
sns.heatmap(temp.corr(), annot= True)
plt.show()

### Looking at the heatmap we can conclude that, addition of the dummy column for counties doesn't affects the overall correlatation. Just the 'swedish' columns add some. Hence we will finalize our columns now 

In [ ]:
col_finalize = ["sex", "age", "country_Sweden"]
X = temp[col_finalize]

In [ ]:
X.head()

In [ ]:
# Splititng  the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42,  test_size = 0.2, stratify = y)

In [ ]:
# Fitting the Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = [KNeighborsClassifier(), GaussianNB(), LogisticRegression(),DecisionTreeClassifier(), RandomForestClassifier()]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
for i in classifier:
    i.fit(X_train, y_train)
    score = cross_val_score(i, X_train, y_train, cv =5)
    y_pred = i.predict(X_test) 
    acc = accuracy_score(y_test, y_pred)
    if(i.__class__.__name__ == "GaussianNB"):
        print(i.__class__.__name__ , "\t\t  Has accuracy of ", round(acc * 100, 3), "\t CV score is ", round(score.mean()*100, 3))
    else:
        print(i.__class__.__name__ , "\t  Has accuracy of ", round(acc * 100, 3), "\t CV score is ", round(score.mean()*100, 3))

In [ ]:
# By looking at the  cv and accuracy_score  we will Logistic Regression for our further anlaysis

In [ ]:
lr = LogisticRegression()
lr.fit (X_train,y_train)
y_predict = lr.predict(X_test)
print("Accuracy score is ",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Tuning the Logistic Regression
from sklearn.model_selection import GridSearchCV
logit  = LogisticRegression()
parm = {
    'C': np.linspace(0.1, 1.0, 10),
    'class_weight': ['balanced','None'],
    'penalty': ['l1', 'l2', 'none'],
    'solver' :['liblinear']
}
gsv = GridSearchCV(logit, parm, scoring = 'roc_auc', n_jobs= -1, cv = 5)
gsv.fit(X_train, y_train)

In [ ]:
gsv.best_score_

### The Logistic Score is much lesser than that of the previous score hence it is not suitable as our assumtions was. Hence we will now try with the RandomForest as the tree based models perform well on the  data

In [ ]:
rfe = RandomForestClassifier(max_depth  = 4, random_state=42)
rfe.fit(X_train, y_train)
y_pred_rfe = rfe.predict(X_test) 
print(accuracy_score(y_test, y_pred_rfe))
print(confusion_matrix(y_test, y_pred_rfe))

###  Without tuning  the RandomForest Performed  well 

In [ ]:
# Tuning the random forest
forest = RandomForestClassifier()
param  = {
    'n_estimators': [100, 300, 500, 800, 1200],
    'max_depth' : [5, 8, 15, 25, 30],
    'min_samples_split' : [2, 5, 10, 15, 100],
    'min_samples_leaf' : [1, 2, 5, 10]
}

In [ ]:
gsv = GridSearchCV(forest, param, cv = 3, verbose = 1, 
                      n_jobs = -1)
gsv.fit(X_train, y_train)

In [ ]:
gsv.best_score_

In [ ]:
final_param = gsv.best_params_
final_param

In [ ]:
forest1 = RandomForestClassifier(**final_param, random_state=42)
forest1.fit(X_train, y_train)
y_pred = forest1.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))
cnf = confusion_matrix(y_test, y_pred)
print(cnf)

In [ ]:
sns.set()
plt.figure(figsize = (6,6))
sns.heatmap(cnf, annot = True, fmt = "2g")

### Here we are The accuracy is 0.86 and it has pretty low misclassification. We can furthur use the XGBoost  but the amount of data is low in amt. Hence we can also improve RFE by adding more Data 

<div>
    <p><i><strong>  UPVOTE IT  :>) </strong> </i></p>
    </div> 